In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [5]:
# Load the data
file_path = 'resources/LoanStats_2019Q1.csv'
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [6]:
# This cell transforms all data to numeric form and prepares the dataframe to be sampled.
# All work here is derived from credit_risk_resampling.ipynb and can be seen there in more detail.
# Both jupyter notebooks load the same starter data, as seen above.

# Create dictionary to manually rename months 
months_num = {
    "Jan-2019": 1,
    "Feb-2019": 2,
    "Mar-2019": 3,
    "Apr-2019": 4,
    "May-2019": 5
}

# copy df before altering
dates_df = df.copy()

# Encode issue_d using dictionary and check
dates_df["issue_d_num"] = dates_df["issue_d"].apply(lambda x: months_num[x])

# Encode next_pymnt_d using dictionary and check
dates_df["pymnt_d_num"] = dates_df["next_pymnt_d"].apply(lambda x: months_num[x])

# Drop original month columns from dataframe
dates_df = dates_df.drop(["issue_d", "next_pymnt_d"], axis=1)

# Convert remaining string values to numeric values
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
loans_df = dates_df.copy()

for column in loans_df:
    if loans_df[column].dtypes == "object":
        loans_df[column] = le.fit_transform(loans_df[column]) 

# Check for non-numeric columns one final time
print(loans_df.dtypes.to_string())
print()
print("Double check with loop (Blank is good): ")
print()
for column in loans_df:
    if (loans_df[column].dtypes != "float64") and (loans_df[column].dtypes != "int64"):
        print('Column Name : ', column, " -- is type -- ", loans_df[column].dtypes)
        print(loans_df[column][0])
        

loan_amnt                     float64
int_rate                      float64
installment                   float64
home_ownership                  int64
annual_inc                    float64
verification_status             int64
loan_status                     int64
pymnt_plan                      int64
dti                           float64
delinq_2yrs                   float64
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
total_acc                     float64
initial_list_status             int64
out_prncp                     float64
out_prncp_inv                 float64
total_pymnt                   float64
total_pymnt_inv               float64
total_rec_prncp               float64
total_rec_int                 float64
total_rec_late_fee            float64
recoveries                    float64
collection_recovery_fee       float64
last_pymnt_amnt               float64
collections_

In [17]:
loans_df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,loan_status,pymnt_plan,dti,delinq_2yrs,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_d_num,pymnt_d_num
0,10500.0,0.1719,375.35,3,66000.0,1,1,0,27.24,0.0,...,0.0,0.0,65687.0,38199.0,2000.0,61987.0,0,0,3,5
1,25000.0,0.2000,929.09,1,105000.0,2,1,0,20.23,0.0,...,1.0,0.0,271427.0,60641.0,41200.0,49197.0,0,0,3,5
2,20000.0,0.2000,529.88,1,56000.0,2,1,0,24.26,0.0,...,0.0,0.0,60644.0,45684.0,7500.0,43144.0,0,0,3,5
3,10000.0,0.1640,353.55,3,92000.0,2,1,0,31.44,0.0,...,1.0,0.0,99506.0,68784.0,19700.0,76506.0,0,0,3,5
4,22000.0,0.1474,520.39,1,52000.0,0,1,0,18.76,0.0,...,0.0,0.0,219750.0,25919.0,27600.0,20000.0,0,0,3,5


# Split the Data into Training and Testing

In [7]:
# Create our features
X = loans_df.drop("loan_status", axis=1)

# Create our target
y = loans_df["loan_status"]

In [8]:
X.describe()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,...,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,issue_d_num,pymnt_d_num
count,68817.000000,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,6.881700e+04,6.881700e+04,68817.000000,6.881700e+04,68817.0,68817.0,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,1.812779,8.821371e+04,0.669994,0.0,21.778153,0.217766,0.497697,...,0.125972,0.0,2.100332e+05,6.133843e+04,29734.128558,5.572240e+04,0.0,0.0,1.726172,4.616839
std,10277.348590,0.048130,288.062432,0.941313,1.155800e+05,0.719105,0.0,20.199244,0.718367,0.758122,...,0.336732,0.0,1.928088e+05,5.738798e+04,26795.394232,5.095845e+04,0.0,0.0,0.743862,0.486161
min,1000.000000,0.060000,30.890000,0.000000,4.000000e+01,0.000000,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,3.600000e+03,2.350000e+02,100.000000,1.270000e+02,0.0,0.0,1.000000,4.000000
25%,9000.000000,0.088100,265.730000,1.000000,5.000000e+04,0.000000,0.0,13.890000,0.000000,0.000000,...,0.000000,0.0,6.697700e+04,2.650300e+04,11600.000000,2.288000e+04,0.0,0.0,1.000000,4.000000
50%,15000.000000,0.118000,404.560000,1.000000,7.300000e+04,1.000000,0.0,19.760000,0.000000,0.000000,...,0.000000,0.0,1.467100e+05,4.535700e+04,22100.000000,4.200000e+04,0.0,0.0,2.000000,5.000000
75%,24000.000000,0.155700,648.100000,3.000000,1.040000e+05,1.000000,0.0,26.660000,0.000000,1.000000,...,0.000000,0.0,3.036400e+05,7.657000e+04,39300.000000,7.249900e+04,0.0,0.0,2.000000,5.000000
max,40000.000000,0.308400,1676.230000,3.000000,8.797500e+06,2.000000,0.0,999.000000,18.000000,5.000000,...,4.000000,0.0,3.292782e+06,1.295455e+06,509400.000000,1.426964e+06,0.0,0.0,3.000000,5.000000


In [10]:
# Check the balance of our target values
y.value_counts()

1    68470
0      347
Name: loan_status, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [12]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=1)
rf_model = rf_model.fit(X_train, y_train)

In [13]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)

balanced_accuracy_score(y_test, y_pred)

0.6644722374931281

In [14]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   28,    57],
       [    8, 17112]])

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.78      0.33      1.00      0.46      0.57      0.31        85
          1       1.00      1.00      0.33      1.00      0.57      0.35     17120

avg / total       1.00      1.00      0.33      1.00      0.57      0.35     17205



In [18]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_

sorted(zip(importances, X.columns), reverse=True)

[(0.08044867856526215, 'last_pymnt_amnt'),
 (0.06739181997232642, 'total_pymnt_inv'),
 (0.06613685978323486, 'total_pymnt'),
 (0.06490047926295679, 'total_rec_prncp'),
 (0.05918430179683875, 'total_rec_int'),
 (0.022832444186295958, 'out_prncp_inv'),
 (0.022467370935344993, 'installment'),
 (0.02167839736614229, 'out_prncp'),
 (0.019473454817892245, 'mo_sin_old_rev_tl_op'),
 (0.017956347834471493, 'total_rec_late_fee'),
 (0.017555476518000403, 'issue_d_num'),
 (0.017250113889876694, 'dti'),
 (0.01572792358830718, 'avg_cur_bal'),
 (0.015581633894245146, 'max_bal_bc'),
 (0.015503318650433444, 'mo_sin_old_il_acct'),
 (0.015164210863948509, 'revol_bal'),
 (0.01487661719255716, 'tot_cur_bal'),
 (0.014497244488938495, 'bc_open_to_buy'),
 (0.014291784039590203, 'loan_amnt'),
 (0.014075726526695278, 'annual_inc'),
 (0.013913619813011011, 'total_bal_il'),
 (0.013842087081343893, 'total_bal_ex_mort'),
 (0.013508553038387576, 'tot_hi_cred_lim'),
 (0.0134253516252677, 'bc_util'),
 (0.0127786902976

### Easy Ensemble AdaBoost Classifier

In [20]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
ee_model

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [21]:
# Calculated the balanced accuracy score
ee_model = ee_model.fit(X_train, y_train)

y_pred_ee = ee_model.predict(X_test)

balanced_accuracy_score(y_test, y_pred_ee)

0.875408878504673

In [22]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_ee)

array([[   68,    17],
       [  842, 16278]])

In [24]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_ee))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.07      0.80      0.95      0.14      0.87      0.75        85
          1       1.00      0.95      0.80      0.97      0.87      0.77     17120

avg / total       0.99      0.95      0.80      0.97      0.87      0.77     17205

